# Import libraries

In [1]:
!pip install keras-tuner -q

     |████████████████████████████████| 135 kB 28.4 MB/s 
     |████████████████████████████████| 1.6 MB 61.1 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
import IPython

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  This is separate from the ipykernel package so we can avoid doing imports until


# Prepare dataset

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

4431872/4422102 [==============================] - 0s 0us/step


# Define model

In [4]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units = hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  
  return model

# Hyper parameter search

In [5]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [6]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [7]:
tuner.search(img_train, label_train, epochs=10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 22s]
val_accuracy: 0.8492000102996826

Best val_accuracy So Far: 0.885200023651123
Total elapsed time: 00h 12m 26s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 416 and the optimal learning rate for the optimizer
is 0.001.



In [8]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4771 - accuracy: 0.8299 - val_loss: 0.4290 - val_accuracy: 0.8432
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3585 - accuracy: 0.8695 - val_loss: 0.4275 - val_accuracy: 0.8466
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3210 - accuracy: 0.8836 - val_loss: 0.3524 - val_accuracy: 0.8738
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2979 - accuracy: 0.8907 - val_loss: 0.3678 - val_accuracy: 0.8703
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2790 - accuracy: 0.8963 - val_loss: 0.3293 - val_accuracy: 0.8829
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2648 - accuracy: 0.9008 - val_loss: 0.3300 - val_accuracy: 0.8836
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2530 - accuracy: 0.9053 - val_loss: 0.3246 - val_accuracy: